<a href="https://colab.research.google.com/github/ruilejin/LLM-review-sentiment/blob/main/mask_language_model_fine_tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

this programm is modified on https://www.youtube.com/watch?v=R6hcxMMOrPE&list=PLqXf3Yqr8OLpYeVp3Nzt1wQiIxvNuXVx_&index=9

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import transformers

In [ ]:
import pandas as pd
import torch
from tqdm.auto import tqdm
from transformers import AdamW
from transformers import AutoTokenizer, AutoModelForMaskedLM
from torch.utils.data import DataLoader, Dataset

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
model = AutoModelForMaskedLM.from_pretrained("roberta-base")

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/review_use.csv',sep=',')
df.head()

,review_content
0,A fantasy adventure that fuses Greek mythology...
1,"Uma Thurman as Medusa, the gorgon with a coiff..."
2,With a top-notch cast and dazzling special eff...
3,Whether audiences will get behind The Lightnin...
4,What's really lacking in The Lightning Thief i...


In [ ]:
data=list(df['review_content'].sample(n=50000,random_state=1)) # sample 50000 review for fine-tuning mlm

In [ ]:
inputs = tokenizer(data,max_length=64,truncation=True,
          padding='max_length',return_tensors='pt'
)

In [ ]:
inputs

To train the model for mask language model, we need 2 things:
1. The input ids which will have a percentage of mask tokens
2. The output labels which will not have the mask tokens that is the input tokens before masking, so before masking the original inputs id, so we will create a copy of input ids, which we will use as our labels later.


In [ ]:
inputs['labels'] = inputs['input_ids'].detach().clone()

In [ ]:
inputs

In [ ]:
inputs.input_ids.shape

In [ ]:
random_tensor=torch.rand(inputs.input_ids.shape)

In [ ]:
random_tensor.shape

Each token that is not a special token has a 15% chance of being masked, so to create that 15% probability for each token, we will use the torch run function to create a float tensor having equal shape of as that of input_ids.

Now we need to give these values a 15% probability of being masked, and we do that by creating a mask array.

( pad_token_id = 1 bos_token_id = 0 eos_token_id = 2 **kwargs )


In [ ]:
masked_tensor=(random_tensor < 0.15)*(inputs.input_ids != 0)*(inputs.input_ids != 1)*(inputs.input_ids != 2)

In [ ]:
masked_tensor

Next get all indices of each row of data which are set to true, that is the mask. Extract the index of ‘TRUE’.
And flatten. And convert to a list.

This will allow us to store index of mask values in each of the row of our data. Append to list.

In [ ]:
selection = []
for i in range(masked_tensor.shape[0]):
  selection.append(torch.flatten(masked_tensor[i].nonzero()).tolist())

In [ ]:
selection[:5]

[[8, 9],
 [],
 [7, 11, 14, 30, 38, 42, 44, 51, 55, 61],
 [2, 11, 12],
 [3, 17, 19, 30]]

Replaces these true value to [MASK] token.

In [ ]:
tokenizer.convert_tokens_to_ids(['<mask>'])

[50264]

In [ ]:
for i in range(len(inputs.input_ids)):
  inputs.input_ids[i,selection[i]] = 50264  # replace input id by <mask> token

In [ ]:
inputs.input_ids

tensor([[    0,   250,  3151,  ...,     1,     1,     1],
        [    0,   734, 26316,  ...,     1,     1,     1],
        [    0, 12444,  5502,  ...,     4,     2,     1],
        ...,
        [    0, 50264,    47,  ...,     1,     1,     1],
        [    0,   713, 50264,  ...,     1,     1,     1],
        [    0, 50264, 50264,  ...,     1,     1,     1]])

In [ ]:
class RottentomatoesDataset(torch.utils.data.Dataset):
  def __init__(self,encodings):
    self.encodings = encodings
  def __len__(self):
    return len(self.encodings.input_ids)
  def __getitem__(self,idx):
    return{key:torch.tensor(val[idx]) for key,val in self.encodings.items()}

In [ ]:
dataset = RottentomatoesDataset(inputs)

In [ ]:
#dataloader = torch.utils.data.DataLoader(dataset,batch_size=32,shuffle=True)
dataloader = torch.utils.data.DataLoader(dataset,batch_size=8,shuffle=True) # 防止内存过载

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
model.to(device)

RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNor

In [ ]:
model.train()

RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNor

In [ ]:
optimizer = AdamW(model.parameters(),lr=1e-5)

In [ ]:
epochs=1
for epoch in range(epochs):
  loop = tqdm(dataloader,leave=True)
  for batch in loop:
    optimizer.zero_grad()
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['labels'].to(device)


    outputs = model(input_ids, attention_mask=attention_mask,
             labels=labels)
    loss = outputs.loss
    loss.backward()
    optimizer.step()

    loop.set_description(f'Epoch{epoch}')
    loop.set_postfix(loss=loss.item())


You can use `model.save_pretrained("<<your-model-name>>")`  to save, and then load in the same way you normally would, eg `BertModel.from_pretrained("<<you-model-name>>")`

In [ ]:
model.save_pretrained("/content/drive/MyDrive/Colab Notebooks/roberta-rottentomatoes")

In [ ]:
AutoModelForMaskedLM.from_pretrained("roberta-rottentomatoes")

RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNor